# Master's Thesis - James Quacinella


# Abstract

**Objectives:** This study will extend an established model for estimating the current living wage in 2015 to the past decade for the purpose of:

* an exploratory analysis trends in the gap between the estimated living wage and the minimum wage
* evaluating any correlation between the living wage gap and other economic metrics, including public funds spent on social services

**Methods:** The original data set for this model is for 2015. This study will extend the data sources of this model into the past to enable trend analysis. Data for economic metrics from public data sources will supplement this data for correlation analysis.


# Methods

## Model

The original model proposed estimated the living wage in terms of 9 variables:

** *basic_needs_budget* ** = *food_cost* + *child_care_cost* + ( *insurance_premiums* + *health_care_costs* ) + *housing_cost* + *transportation_cost* + *other_necessities_cost*

** *living_wage* ** = *basic_needs_budget* + ( *basic_needs_budget* \* *tax_rate* )

## Data Sources

The following data sources are used to find estimates of the model variables:

* The food cost is estimated from data from the USDA’s low-cost food plan national average in June 2014.
* Child care is based off state-level estimates published by the National Association of Child Care Resource and Referral Agencies.
* Insurance costs are based on the insurance component of the 2013 Medical Expenditure Panel Survey.
* Housing costs are estimated from the HUD Fair Market Rents (FMR) estimates
* Other variables are pulled from the 2014 Bureau of Labor Statistics Consumer Expenditure Survey.

These data sets extend into the past, allowing for calculating the model for years past. The data will also have to be adjusted for inflation 6.

## Analytic Approach

First, data will be gathered from the data sources of the original model but will be extended into the past. The methodology followed by the model will be replicated to come up with a data set representing estimates of the living wage across time. After the data set is prepared, the trend of the living wage as compared to minimum wage can be examined. Has the gap increased or decreased over time, and at what rate? Have certain areas seen larger than average increases or decreases in this gap? 

Once preliminary trend analysis is done, this data set will be analyzed in comparison to other economic trends to see if any interesting correlations can be found. Correlations to GDP growth rate and the national rate of unemployment can be made, but the primary investigation will be to see if the living wage gap correlates to national spending on SNAP (Food stamps). In other words, we will see if there is any (potentially time lagged) relationship between the living wage gap and how much the United States needs to spend to support those who cannot make ends meet. A relationship here can potentially indicate that shrinking this gap could lower public expenditures.


## Presentation Of Results

Results will be presented for both parts of the data analysis. For studying the living wage gap trends, this report will present graphs of time series, aggregated in different ways, of the living wage as well as the living wage gap. Some of these time series will be presented along side data on public expenditures on SNAP to visually inspect for correlations.

## Background / Sources

- Glasmeier AK, Nadeau CA, Schultheis E: LIVING WAGE CALCULATOR User’s Guide / Technical Notes 2014 Update
- USDA low-cost food plan, June, 2014
- Child Care in America 2014 State fact sheets
- 2013 Medical Expenditure Panel Survey Available
- Consumer Expenditure Survey
- Inflation Calculator

------

------

------

# Pre-Data Collection

Lets do all of our imports now:

In [4]:
import numpy as np
from prettytable import PrettyTable
from IPython.core.display import HTML
from collections import OrderedDict, defaultdict
from bs4 import BeautifulSoup
import os
from pprint import pprint
import pandas as pd

PROJECT_PATH = "/home/james/Development/Masters/Thesis" # Path to project files on my local computer

def caption(msg, tablenum):
    ''' Help convert text into suitable table caption. '''
    return "<br><b>Table %d - %s</b>" % (tablenum, msg)

Lets setup some inflation multipliers:

**TODO** Fill in more values for updated multipliers

In [151]:
# Multiply a dollar value to get the equivalent 2014 dollars
# Original numbers from model; used to confirm model
inflation_multipliers = {
    2010: 1.092609, 
    2011: 1.059176,
    2012: 1.037701,
    2013: 1.022721,
    2014: 1.0
}

# Updated inflation numbers should scale to 2015 dollars
updated_inflation_multipliers = {
    2001: 1.0,
    2002: 1.0,
    2003: 1.0,
    2004: 1.0,
    2005: 1.0,
    2006: 1.0,
    2007: 1.0,
    2008: 1.0,
    2009: 1.0, # 1.0's just to have a value for this key ...
    2010: 1.092609, 
    2011: 1.059176,
    2012: 1.037701,
    2013: 1.022721,
    2014: 1.0
}

Global identifiers used throught the project:

In [5]:
# Constants used to refer to US regions
REGION_EAST = 'east'
REGION_MIDWEST = 'midwest'
REGION_SOUTH = 'south'
REGION_WEST = 'west'

Lets setup regional differences for the food data:

In [6]:
# Multiply price of food by regional multipler to get better estimate of food costs
food_regional_multipliers = {
    REGION_EAST: 0.08,
    REGION_WEST: 0.11,
    REGION_SOUTH: -0.07,
    REGION_MIDWEST: -0.05,
}

------

------

------

#  Data Collection

The following sections will outline how I gathered the data for the various model parameters as well as other data we need to calculate their values. The original model was made for 2014 data and extending this data to the past means we need to be careful that any changes in the underlying data methodology of these parameters needs to be noted.

## Data Sources

### Consumer Expenditure Report

Wget commands used to get the Consumer Expenditure Reports:

In [ ]:
# Get CEX for 2013 and 2014 (XLSX format)
for i in `seq 2013 2014`; do wget http://www.bls.gov/cex/$i/aggregate/cusize.xlsx -O ${i}_cex.xlsx; done

# Get CEX for 2004 - 2012 (XLS format)
for i in `seq 2004 2012`; do wget http://www.bls.gov/cex/$i/aggregate/cusize.xls -O ${i}_cex.xls; done

# Get CEX for 2001 to 2003 (TXT format)
for i in `seq 2001 2003`; do wget http://www.bls.gov/cex/aggregate/$i/cusize.txt -O ${i}_cex.txt; done


# Get CEX region for 2013 and 2014 (XLSX format)
for i in `seq 2013 2014`; do wget http://www.bls.gov/cex/$i/aggregate/region.xlsx -O ${i}_region_cex.xlsx; done

# Get CEX region for 2004 - 2012 (XLS format)
for i in `seq 2004 2012`; do wget http://www.bls.gov/cex/$i/aggregate/region.xls -O ${i}_region_cex.xls; done

# Get CEX region for 2001 to 2003 (TXT format)
for i in `seq 2001 2003`; do wget http://www.bls.gov/cex/aggregate/$i/region.txt -O ${i}_region_cex.txt; done

### USDA Food Plans

Wget commands used to gather data files:

In [ ]:
# Change command to get '10 - '15
for i in {1..9}; do  wget http://www.cnpp.usda.gov/sites/default/files/usda_food_plans_cost_of_food/CostofFoodJun0$i.pdf; done

### Free Market Rent Data From HUD

Below are the wget commands for getting the FMR data

#### TODO 

* Extract counties -> state -> region mapping

In [ ]:
for i in `seq 2014 2015`; do wget http://www.huduser.gov/portal/datasets/fmr/fmr${i}f/FY${i}_4050_RevFinal.xls -O fmr${i}.xlsx; done
for i in `seq 2010 2013`; do wget http://www.huduser.gov/portal/datasets/fmr/fmr${i}f/FY${i}_4050_Final.xls -O fmr${i}.xlsx; done
for i in `seq 2009 2009`; do wget http://www.huduser.gov/portal/datasets/fmr/fmr${i}r/FY${i}_4050_Rev_Final.xls -O fmr${i}.xlsx; done

# GRRRR
wget http://www.huduser.gov/portal/datasets/fmr/fmr2008r/FMR_county_fy2008r_rdds.xls
wget http://www.huduser.gov/portal/datasets/fmr/fmr2007f/FY2007F_County_Town.xls
wget http://www.huduser.gov/portal/datasets/fmr/fmr2006r/FY2006_County_Town.xls
wget http://www.huduser.gov/portal/datasets/fmr/fmr2005r/Revised_FY2005_CntLevel.xls
wget http://www.huduser.gov/portal/datasets/FMR/FMR2004F/FMR2004F_County.xls
wget http://www.huduser.gov/portal/datasets/fmr/FMR2003F_County.xls
wget http://www.huduser.gov/portal/datasets/fmr/FMR2002F.xls

In [23]:
# Counties dict will map county ID to useful infomation, mostly region
counties = { }

### Medical Expenditure Panel Survey from the Agency for Healthcare Research and Quality

Below are the wget commands used to download this data. This data will have to be further parsed from HTML.

In [ ]:
# Need to figure out what to do prior to 2006
for i in `seq 2006 2014`; do 
    wget -O ${i}_txc1.html http://meps.ahrq.gov/mepsweb/data_stats/summ_tables/insr/state/series_10/${i}/txc1.htm; 
done 


---

---

---

## Model Variable: Food

Data for the food calculations have been successfully downloaded in PDF form. The main way to calculate this is, from the PDF:

>Adult  food  consumption  costs  are  estimated  by  averaging  the  low - cost  plan  food  costs for  males  and  females  between  19  and  50

Note, we add 20% to the values from the data sheets, since the notes on all published PDFs from the USDA state to add 20% to the listed values for individuals since:

>The costs given are for individuals in 4-person families. For individuals in other size families, the following adjustments are suggested: 1-person—add 20 percent; ...

The notes for the model also state that regional weights are applied to give a better estimate for food costs across the nation. The result of this section are values fo 2014 that match exactly tot he data given on the model website, so I am confident the implementation of the methodology below is correct.

### Notes: Change of Methodology

In 2006, the data from the USDA changed the age ranges for their healthy meal cost calculations. The differences in range are minimal and should not effect overall estimations.

In [67]:
# The base food cost (not regionally weighed) for nation (data pulled manually from PDFs)
national_monthly_food_cost_per_year = {
    2014: {"base": np.average([241.50, 209.80])},
    2013: {"base": np.average([234.60, 203.70])},
    2012: {"base": np.average([234.00, 203.00])},
    2011: {"base": np.average([226.80, 196.90])},
    2010: {"base": np.average([216.30, 187.70])},
    2009: {"base": np.average([216.50, 187.90])},
    2008: {"base": np.average([216.90, 189.60])},
    2007: {"base": np.average([200.20, 174.10])},
    2006: {"base": np.average([189.70, 164.80])},
    2005: {"base": np.average([186.20, 162.10])},
    2004: {"base": np.average([183.10, 159.50])},
    2003: {"base": np.average([174.20, 151.70])},
    2002: {"base": np.average([170.30, 148.60])},
    2001: {"base": np.average([166.80, 145.60])},
}

# Create ordered dict to make sure we process things in order
national_monthly_food_cost_per_year = OrderedDict(sorted(national_monthly_food_cost_per_year.items(), 
                                                        key=lambda t: t[0]))

# Adjust the data according to notes above
for year in national_monthly_food_cost_per_year:
    # Inflation and 20% adjustment
    national_monthly_food_cost_per_year[year]["base"] = \
        national_monthly_food_cost_per_year[year]["base"] * 1.20 * updated_inflation_multipliers[year]

    # Regional adjustment
    national_monthly_food_cost_per_year[year]["regional"] = { }
    for region in food_regional_multipliers:
        national_monthly_food_cost_per_year[year]["regional"][region] = \
            national_monthly_food_cost_per_year[year]["base"] * (1 + food_regional_multipliers[region])

In yearly form:

In [149]:
# Print it nicely in yearly costs
pt = PrettyTable()
pt.add_column("Year", national_monthly_food_cost_per_year.keys())
pt.add_column("Food Cost (per year)", [np.round(x["base"] * 12) for x in national_monthly_food_cost_per_year.values()])
for region in food_regional_multipliers:
    pt.add_column("Food Cost (%s)" % region, [np.round(x["regional"][region] * 12) for x in national_monthly_food_cost_per_year.values()])

# Print as HTML
HTML(pt.get_html_string() + caption("Food Data Loaded from USDA Pricing on Meals", 1))

Year,Food Cost (per year),Food Cost (west),Food Cost (east),Food Cost (midwest),Food Cost (south)
2001,2249.0,2497.0,2429.0,2137.0,2092.0
2002,2296.0,2549.0,2480.0,2181.0,2135.0
2003,2346.0,2605.0,2534.0,2229.0,2182.0
2004,2467.0,2738.0,2664.0,2343.0,2294.0
2005,2508.0,2784.0,2708.0,2382.0,2332.0
2006,2552.0,2833.0,2757.0,2425.0,2374.0
2007,2695.0,2991.0,2911.0,2560.0,2506.0
2008,2927.0,3249.0,3161.0,2780.0,2722.0
2009,2912.0,3232.0,3145.0,2766.0,2708.0
2010,3178.0,3528.0,3432.0,3019.0,2956.0


## Model Variable: Transportation Cost

Looking at the (1) Cars and trucks (used), (2) gasoline and motor oil, (3) other vehicle expenses, and (4)  public  transportation fields under "Transportation" in the 2014 Consumer Expenditure Report, we can pull out information from each to model the claculation done in the original model. For each sub-variable, we get the amount of money (in millions) and the percentgae of that that single adults spend. After multiple those numbers (accounting for units) and dividiing by the total number of single adults in the survey gives us a mean total cost per adult.

The original model takes into account regional drift by scaling based on each regions. NOTE: See todo in this section

Since this data reflects conditions in 2013, we account for inflation to get the 2014 estimate that is produced in the original model.

### TODO:

* Figure out how to do regional differences correctly. Emailed model creator for clarification

In [55]:
# Transportation data from 2014 survey is for year 2013, etc
cex = {
    2012: {
        "single_adults": 37770.0,
        "transport": {
            "used_car": 209764.0,
            "gasoline": 328170.0,
            "other_vehicle": 324668.0,
            "public": 67486.0,
            "used_car_percent": 0.152,
            "gasoline_percent": 0.158,
            "other_vehicle_percent": 0.191,
            "public_percent": 0.174,
            "regional": {
                REGION_EAST:   16.4 / 17.6,  
                REGION_MIDWEST: 18.0 / 17.6,
                REGION_SOUTH: 18.9 / 17.6,
                REGION_WEST: 16.5 / 17.6,
            }
        }

    },
    2013: {
        "single_adults": 37884.0,
        "transport": {
            "used_car": 214524.0,
            "gasoline": 313481.0,
            "other_vehicle": 345454.0,
            "public": 73842.0,
            "used_car_percent": 0.146,
            "gasoline_percent": 0.157,
            "other_vehicle_percent": 0.163,
            "public_percent": 0.172,
            "regional": {
                REGION_EAST: 15.7 / 17.0,     # 0.923
                REGION_MIDWEST: 16.9 / 17.0,  # 0.994
                REGION_SOUTH: 18.3 / 17.0,    # 1.076
                REGION_WEST: 16.1 / 17.0,     # 0.947
            }
        }
    },
}

# Ideal numbers from model
ideal_transport_2013 = (3764, 4569, 4697, 4054)

# Base price for transport
transportation_costs = defaultdict(dict)

for year in cex:
    transportation_costs[year]["base"] = \
        (1000000 * ((cex[year]["transport"]["used_car"] * cex[year]["transport"]["used_car_percent"]) + \
                    (cex[year]["transport"]["gasoline"] * cex[year]["transport"]["gasoline_percent"]) + \
                    (cex[year]["transport"]["other_vehicle"] * cex[year]["transport"]["other_vehicle_percent"] ) + \
                    (cex[year]["transport"]["public"] * cex[year]["transport"]["public_percent"] )) /  float(cex[year]["single_adults"] * 1000) ) * inflation_multipliers[year]

    # Account for regional drift
    for region in cex[year]["transport"]["regional"]:
        transportation_costs[year][region] = transportation_costs[year]["base"] * cex[year]["transport"]["regional"][region]

transportation_costs["2014_ideal"]["base"] = 0.0
transportation_costs["2014_ideal"][REGION_EAST] = ideal_transport_2013[0]
transportation_costs["2014_ideal"][REGION_MIDWEST] = ideal_transport_2013[1]
transportation_costs["2014_ideal"][REGION_SOUTH] = ideal_transport_2013[2]
transportation_costs["2014_ideal"][REGION_WEST] = ideal_transport_2013[3]

# Print it nicely
errors = []
pt = PrettyTable()
pt.add_column("Year", transportation_costs.keys())
for region in sorted(transportation_costs[2013].keys()):
    data = [ transportation_costs[year][region] for year in transportation_costs  ]
    pt.add_column("Trans Cost (%s)" % region, data)
    errors.append(transportation_costs["2014_ideal"][region] - data[-2])

print(sum([np.abs(error) for error in errors]))

# Print as HTML
HTML(pt.get_html_string())

5209.92768399


Year,Trans Cost (base),Trans Cost (east),Trans Cost (midwest),Trans Cost (south),Trans Cost (west)
2012,4326.89007326,4031.87484099,4425.22848402,4646.48990822,4056.45944368
2013,4037.18458744,3728.45870723,4013.43644281,4345.91046766,3823.45128575
2014_ideal,0.0,3764,4569,4697,4054


### Testing theory about regional difference

In [123]:
# Order: NE, MW, S, W
used_car_rations = (2.5 / 3.2, 3.5 / 3.2, 3.5 / 3.2, 2.9 / 3.2)
gas_rations = (3.8 / 4.6, 4.7 / 4.6, 5.2 / 4.6, 4.5 / 4.6)
other_rations = (5.2 / 5.1, 5.0  / 5.1, 5.1 / 5.1,  5.1 / 5.1)
public_rations = (1.6/1.1,  0.9/1.1,  0.8/1.1, 1.2/1.1)

error = []
for region in range(4):
    val = (1000000 * 
         ( (
            (cex[2013]["transport"]["used_car"] * cex[2013]["transport"]["used_car_percent"] * used_car_rations[region]) + \
            (cex[2013]["transport"]["gasoline"] * cex[2013]["transport"]["gasoline_percent"] * gas_rations[region]) + \
            (cex[2013]["transport"]["other_vehicle"] * cex[2013]["transport"]["other_vehicle_percent"] * other_rations[region]) + \
            (cex[2013]["transport"]["public"] * cex[2013]["transport"]["public_percent"] * public_rations[region])
        ) /  (float(cex[2013]["single_adults"] * 1000)) ) * inflation_multipliers[2013])
    errors.append( val - ideal_transport_2013[region] )

print(sum([np.abs(error) for error in errors]))

7516.27558175


In [147]:
# calculate regional diff values from aggregated data (since 'combined' only goes back to 2012)
print 1/ (6790803*1000000*20.1 / (1152035*1000000*18.6))
print 1/ (6790803*1000000*21.7 / (1152035*1000000*21.7))
print 1/ (6790803*1000000*34.3 / (1152035*1000000*37.1))
print 1/ (6790803*1000000*23.9 / (1152035*1000000*22.6))


print 1152035/6790803.0



0.15698618246
0.169646358465
0.183495040788
0.160418732272
0.169646358465


## Model Variable: Child Care Cost

Manually download PDFs from ChildCareAware.org. Sadly, they only go back to 2010. I can now either:

* have to find other estimates of child care costs from pre-2010 (prefered)
* check if the Consumer Expenditure Survey has data on this
* impute the data (dont think this is a good idea)
* limit the analysis going back to 2010 (which seems limiting since other data, like the Consumer Expenditure Survey in 2014 provides 2013 data and that is the latest currently).

Currently I am only focusing on modeling costs for a single adult (an assumption I made early on) since I am interested in trends, and the other 'family configurations' are just linear combinations of the costs for one adult and for one child. However if I wanted to extend the numbers for 1 adult + 1 child, I would have to look into this further. For now I'll move on.

## Model Variable: Insurance Premiums

The model uses data from the Medical Expenditure Panel Survey from the Agency for Healthcare Research and Quality (searchable [here](http://meps.ahrq.gov/mepsweb/data_stats/quick_tables_search.jsp?component=2&subcomponent=2)). Specifically, the model assumes a single adult's insurance costs are best estimated from Table X.C.1 Employee contribution distributions (in dollars) for private-sector employees enrolled in single coverage. This survey gives the mean cost for a single adult per state.

Below is code on processing each html file.

**PROBLEMS**

* One problem is that in 2007 this survey was not done.
    * Linearly impute data from 2006 and 2008? Seems resonable if we can assume that costs tend to go up every year and not go down, which makes it seem likely that 2007 values would be bounded by the previous and next year
* Another problem is that this portion of the survey started in 2006
    * Find another data source for pre 2006?
    * Limit analysis to 2006 - 2014, which is nearly a decade

In [126]:
# Process HTML files with BeautifulSoup
insurance_costs = {}
insurance_costs_path = os.path.join(PROJECT_PATH, "data/insurance")

# Loop thru all the files
for filename in os.listdir(insurance_costs_path):
    states = {}
    
    # File is for what year?
    year = int(filename.split('_')[0])
    
    # Open file
    full_filename = os.path.join(insurance_costs_path, filename)
    f = open(full_filename, "r")
    
    # Import into BeautifulSoup
    data = f.readlines()
    soup = BeautifulSoup(''.join(data))

    # Works for years 2010 - 2014
    if year in range(2010, 2015):
        for tr in soup.find_all('tr'):
            # State is located in the TR element
            state = tr.get_text().split("\n")[1].lower().strip()
            
            # Find the data, but if you can't, skip it
            td = tr.find_all('td')
            value = None
            if td: 
                try:
                    value = float(td[0].get_text().strip().replace(",", ""))
                    
                    # Account for inflation and round up
                    value = np.round(value * updated_inflation_multipliers[year])
                except ValueError as e:
                    continue

            # We need to stop processing after the first chunk or if we couldnt get a value
            if state not in states and value:
                states[state] = value
    # Works for 2006, 2008 - 2009
    else:
        for tr in soup.find_all('tr'):
            td = tr.find_all('td')

            value = None
            if len(td) > 2: 
                # Same as above, but state is fist TD, not in TR
                state = td[0].get_text().lower().strip()
                try:
                    value = float(td[1].get_text().strip().replace(",", ""))
                    
                    # Account for inflation
                    value = np.round(value * updated_inflation_multipliers[year])
                except ValueError as e:
                    continue

            if state not in states and value:
                states[state] = value

    # Add data from file to global dict
    insurance_costs[year] = states

In [150]:
# Print results
pt = PrettyTable()

# Setup column names but skip DC and United states total
pt.field_names = ["Year"] + [state for state in sorted(insurance_costs[2013].keys()) \
                             if "district" not in state and 'united' not in state]

# Add rows from dict we have on insurance costs
for year in insurance_costs.keys():
    pt.add_row([year] + [insurance_costs[year][state] for state in sorted(insurance_costs[year].keys()) \
                         if "district" not in state and 'united' not in state])

# Print as HTML
HTML(pt.get_html_string() + caption("Data for insurance costs across the united states per year, adjusted for inflation", 2))

Year,alabama,alaska,arizona,arkansas,california,colorado,connecticut,delaware,florida,georgia,hawaii,idaho,illinois,indiana,iowa,kansas,kentucky,louisiana,maine,maryland,massachusetts,michigan,minnesota,mississippi,missouri,montana,nebraska,nevada,new hampshire,new jersey,new mexico,new york,north carolina,north dakota,ohio,oklahoma,oregon,pennsylvania,rhode island,south carolina,south dakota,tennessee,texas,utah,vermont,virginia,washington,west virginia,wisconsin,wyoming
2006,891.0,714.0,803.0,699.0,658.0,717.0,862.0,735.0,860.0,862.0,366.0,565.0,822.0,833.0,784.0,765.0,691.0,755.0,1072.0,898.0,1011.0,682.0,810.0,727.0,703.0,598.0,873.0,551.0,1004.0,902.0,726.0,965.0,704.0,675.0,781.0,650.0,547.0,881.0,862.0,810.0,718.0,745.0,728.0,826.0,738.0,981.0,623.0,825.0,885.0,655.0
2008,959.0,814.0,811.0,781.0,741.0,998.0,992.0,885.0,1065.0,972.0,451.0,476.0,954.0,950.0,756.0,807.0,806.0,868.0,1054.0,964.0,1110.0,735.0,891.0,749.0,956.0,583.0,1010.0,863.0,1264.0,1033.0,950.0,947.0,827.0,754.0,885.0,787.0,612.0,852.0,1050.0,849.0,887.0,914.0,844.0,752.0,986.0,988.0,569.0,1049.0,1069.0,717.0
2009,1025.0,842.0,851.0,750.0,795.0,971.0,1082.0,1101.0,969.0,963.0,461.0,762.0,1008.0,1070.0,855.0,976.0,1000.0,956.0,981.0,1105.0,1321.0,946.0,994.0,994.0,999.0,768.0,873.0,842.0,1087.0,1045.0,934.0,1075.0,998.0,860.0,1065.0,815.0,627.0,917.0,1207.0,898.0,890.0,1010.0,991.0,772.0,1008.0,1060.0,640.0,1085.0,1011.0,729.0
2010,1193.0,909.0,974.0,967.0,1145.0,965.0,1348.0,1289.0,1172.0,1054.0,476.0,909.0,1224.0,1231.0,1016.0,1011.0,968.0,1356.0,1319.0,1180.0,1311.0,1039.0,1118.0,1125.0,1054.0,1140.0,1184.0,838.0,1187.0,1200.0,1288.0,1187.0,1012.0,974.0,1040.0,1140.0,927.0,1042.0,1253.0,1099.0,1036.0,1060.0,1132.0,1187.0,1201.0,1217.0,815.0,1019.0,1283.0,876.0
2011,1195.0,1146.0,1209.0,1028.0,1032.0,1122.0,1273.0,1183.0,1202.0,1314.0,578.0,936.0,1278.0,1098.0,1142.0,1048.0,1174.0,1289.0,1179.0,1310.0,1523.0,1166.0,1151.0,1045.0,1223.0,872.0,1111.0,1093.0,1310.0,1281.0,1346.0,1218.0,1124.0,1045.0,1193.0,1096.0,925.0,1127.0,1470.0,1299.0,1191.0,1092.0,1058.0,1013.0,1293.0,1145.0,917.0,1049.0,1161.0,928.0
2012,1279.0,1208.0,1200.0,1024.0,1035.0,1148.0,1368.0,1373.0,1213.0,1160.0,535.0,962.0,1190.0,1201.0,1234.0,1340.0,1149.0,1118.0,1128.0,1157.0,1566.0,1099.0,1258.0,1117.0,1175.0,826.0,1183.0,1063.0,1308.0,1269.0,1263.0,1301.0,1033.0,1010.0,1276.0,1137.0,871.0,1102.0,1385.0,1192.0,1260.0,1080.0,1051.0,1177.0,1289.0,1306.0,910.0,1151.0,1320.0,1111.0
2013,1410.0,1102.0,1102.0,978.0,1116.0,1188.0,1536.0,1459.0,1440.0,1247.0,441.0,997.0,1331.0,1160.0,1224.0,1106.0,1243.0,1242.0,1144.0,1338.0,1683.0,1178.0,1260.0,1122.0,1060.0,902.0,1190.0,1332.0,1447.0,1282.0,1142.0,1320.0,1088.0,992.0,1077.0,1086.0,822.0,1098.0,1433.0,1163.0,1378.0,1194.0,1161.0,1114.0,1197.0,1272.0,695.0,1076.0,1248.0,1083.0
2014,1362.0,1286.0,1096.0,958.0,1129.0,1244.0,1305.0,1237.0,1394.0,1203.0,460.0,1039.0,1306.0,1347.0,1353.0,1072.0,1314.0,1302.0,1176.0,1422.0,1588.0,1315.0,1217.0,1154.0,1243.0,1024.0,1322.0,1204.0,1481.0,1293.0,1354.0,1223.0,1151.0,1136.0,1260.0,1154.0,914.0,1141.0,1459.0,1332.0,1213.0,1409.0,1211.0,1297.0,1281.0,1296.0,937.0,1297.0,1257.0,1139.0


## Model Variable: Health Care Costs

Calculated from the CEX data from above, essentially done once regional differencing is done

### TODO

* Complete data load once regional differences are figured out

## Model Variable: Housing Cost

Definition from the model:

> We assumed  that  a  one  adult  family  would  rent a  single occupancy unit (zero bedrooms) for an individual adult household, that a two adult family would  rent a one bedroom apartment,

### TODO

* download all data (DONE, in above section on data downloading)
* figure out how to extract what we need from ecah XLS and extract CSV files (DONE)
* import CSV files into pandas data frames (DONE for 2009-2014)
* figure out multi level index for data
* figure out how to get estimate for HUD areas

In [6]:
# Fair Market Rent data
fmr_data = { }

# For now, only loading 2009 - 2014
for year in range(2009, 2015):
    with open(PROJECT_PATH + "/data/fmr/fmr%d.csv" % year, 'rb') as csvfile:
        fmr_data[year] = pd.read_csv(csvfile)
        try:
            fmr_data[year] = fmr_data[year][["fmr0", "county", "CouSub", "countyname", "fips2000", "fips2010", "pop2010", "state_alpha"]]
        except:
            fmr_data[year] = fmr_data[year][["fmr0", "county", "CouSub", "countyname", "FIPS", "pop2000", "state_alpha"]]

# Print example
fmr_data[2009]

,fmr0,county,CouSub,countyname,FIPS,pop2000,state_alpha
0,530,1,99999,Autauga County,100199999,43671,AL
1,512,3,99999,Baldwin County,100399999,140415,AL
2,430,5,99999,Barbour County,100599999,29038,AL
3,563,7,99999,Bibb County,100799999,20826,AL
4,563,9,99999,Blount County,100999999,51024,AL
5,381,11,99999,Bullock County,101199999,11714,AL
6,381,13,99999,Butler County,101399999,21399,AL
7,410,15,99999,Calhoun County,101599999,112249,AL
8,427,17,99999,Chambers County,101799999,36583,AL
9,446,19,99999,Cherokee County,101999999,23988,AL


## Model Variable: Other Necessities Cost

Calculated from the CEX data from above, essentially done once regional differencing is done

>   Expenditures for other necessities are based on 
2013 data by household size  from  the  2014 Bureau  of  Labor  Statistics  Consumer  Expenditure  Survey
including: (1) Apparel  and  services,  (2)  Housekeeping  supplies,  (3)  Personal  care  products  and  services, 
(4)  Reading, and (5) Miscellaneous.  These costs were further adjusted for regional differences using annual  expenditure  shares  reported  by  region



### TODO

* Complete data load once regional differences are figured out

In [48]:
# Update cex dictionary with values for other variable
cex[2013].update( 
    {
        "single_adults": 37884.0,
        "other": {
            "apparel": 226385.0, 
            "housekeeping": 80097.0,
            "personal_care": 81837.0,
            "reading": 13086,
            "misc": 99290,
            
            "apparel_percent": 0.13,
            "housekeeping_percent": 0.164,
            "personal_care_percent": 0.182,
            "reading_percent": 0.205,
            "misc_percent": 0.228,
            
            "apparel_region": [ x / 3.3 for x in (3.3, 3.6, 3.2, 3.3)],
            "housekeeping_region": [ x / 1.2 for x in (1.0, 1.4, 1.2, 1.1 )],
            "personal_care_region": [ x / 1.2 for x in (1.2, 1.3, 1.2, 1.2 )],
            "reading_region": [ x / 1.0 for x in (1,1,1,1)],
            "misc_region": [ x / 1.5 for x in (1.4, 1.5, 1.3, 1.6)],
        }
    }
)

# Values for 'other' from county webpages
ideal_other_2013 = (2096, 2127, 2253, 2284)

for region in range(4):
    val = (1000000 * 
         ( (
            (cex[2013]["other"]["apparel"] * cex[2013]["other"]["apparel_percent"] * cex[2013]["other"]["apparel_region"][region]) + \
            (cex[2013]["other"]["housekeeping"] * cex[2013]["other"]["housekeeping_percent"] * cex[2013]["other"]["housekeeping_region"][region]) + \
            (cex[2013]["other"]["personal_care"] * cex[2013]["other"]["personal_care_percent"] * cex[2013]["other"]["personal_care_region"][region]) + \
            (cex[2013]["other"]["reading"] * cex[2013]["other"]["reading_percent"] * cex[2013]["other"]["reading_region"][region]) + \
            (cex[2013]["other"]["misc"] * cex[2013]["other"]["misc_percent"] * cex[2013]["other"]["misc_region"][region])
        ) /  (float(cex[2013]["single_adults"] * 1000)) ) * inflation_multipliers[2013])

    # Print difference between calc and data from website
    print "Diff for region %d: %f %f" % (region, val, ideal_other_2013[region])

Diff for region 0: 2134.921071 2096.000000
Diff for region 1: 2399.604462 2127.000000
Diff for region 2: 2129.205730 2253.000000
Diff for region 3: 2245.958136 2284.000000


In [42]:
float(cex[2013]["single_adults"] * 1000)

37884000.0

## Taxes Data

## Minimum Wage or Mediun Wage per County or State

-----

-----

-----

## Creating Final Merged Data Frame

Take all data loaded in prior into a multi-level index data frame

## Introductory Analysis

Create visualizations on:

* Find national mean living wage gap, plot it over time
* Look at distributions over states of living wage gap over time (facet grid, each graph is a state showing gap over time)
* Seperate counties based on race and find national means of gap per year


## Correlations with Economic Metrics

* motion chart of states, x = gap, y = life exp, debt levels